# CoursewareHub環境の削除

---

構築した環境を削除します。

## パラメータの指定

### VCCアクセストークンの入力

VCノードを削除するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### グループ名

削除対象となるVCPのUnitGroup名を指定します。

対象となり得るUnitGroupを確認するために group_varsファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

 UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

## 構築環境の削除

### VCノードの削除

起動したVCノードを削除します。

現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

現在のVCノードの状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(ugroup_name)
ugroup.df_nodes()

スケジュール設定が行われている場合はノードを停止する前にsystemdタイマーを停止します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if any(name in gvars for name in ["vcnode_schedule", "schedule"]):
    !ansible {ugroup_name}_manager -b -m systemd \
        -a 'name=cwh-worker-node.timer enabled=false state=stopped'

まず worker の VCノードを削除します。

> workerのVCノードがない場合は何もしません。

In [ ]:
worker_unit = ugroup.get_unit('worker')
if worker_unit:
    worker_unit.delete_nodes(num_delete_nodes=len(worker_unit.find_nodes()))

マスターのVCノードを削除します。

In [ ]:
manager_unit = ugroup.get_unit('manager')
if manager_unit:
    manager_unit.delete_nodes()

UnitGroupに属する全てのVCノードとUnitGroupの削除を行います。

In [ ]:
ugroup.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### VCディスクの削除

NFS用のVCディスクを削除します。

> VCディスクを作成していない場合は、何もしません。

現在の状態を確認します。

In [ ]:
from IPython.display import display
ug_disk = vcp.get_ugroup(ugroup_name + '_disk')
if ug_disk:
    display(ug_disk.df_nodes())

VCディスクを削除します。

In [ ]:
if ug_disk:
    ug_disk.cleanup()

削除後のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### /etc/exportsの更新

削除対象が構成3の場合、削除したVCノードをNFSサーバの`/etc/exports` から削除する必要があります。

> 対象が構成3以外の場合は、この節では何も実行しません。

NFSサーバの `/etc/exports` から、削除したVCノードに対応する設定を削除します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if 'nfs_target' in gvars and gvars['nfs_target'] != gvars['ugroup_name']:
    !ansible {gvars['nfs_target']} -b -m file -a 'path=/etc/exports.d/{ugroup_name}.exports state=absent'
    !ansible {gvars['nfs_target']} -b -a 'exportfs -r -v'

## Ansible設定のクリア

削除した環境に対応するAnsibleの設定をクリアします。

### group_varsファイル

group_varsファイルをリネームします。

In [ ]:
!mv group_vars/{ugroup_name} group_vars/.{ugroup_name}.bak

### インベントリ

インベントリから削除対象のグループを除去します。

In [ ]:
 %run scripts/group.py
remove_group_from_inventory_yml(ugroup_name)

# 削除前との差分を表示します。
! ! diff -u inventory.yml.bak inventory.yml